<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/010_Mask2Former.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# ==========================================
# 1. SETUP INICIAL
# ==========================================
import torch
import numpy as np
import cv2
import json
import time
import os
from datetime import datetime
from pathlib import Path
import hashlib
from dataclasses import dataclass
from typing import Dict, List, Any, Optional

from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
from tqdm import tqdm

# Configuración
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
@dataclass
class Config:
    """Configuración centralizada"""
    DATASET_PATH: Path = Path("/content/drive/MyDrive/TFM/mask2former/imagenes")
    OUTPUT_PATH: Path = Path("/content/drive/MyDrive/TFM/mask2former/resultados")

    # Umbrales predefinidos
    UMBRALES = {
        'conservador': [0.3, 0.5, 0.7],
        'normal': [0.1, 0.3, 0.5],
        'agresivo': [0.01, 0.05, 0.1, 0.3],
        'ultra': [0.001, 0.01, 0.05, 0.1]
    }

    MODELOS = [
        "facebook/mask2former-swin-large-coco-instance",
        "facebook/mask2former-swin-base-ade-semantic",
        "facebook/mask2former-swin-small-coco-instance"
    ]

In [28]:
config = Config()
os.makedirs(config.OUTPUT_PATH, exist_ok=True)
print("Setup completado")

Setup completado


In [29]:
# ==========================================
#  2: UTILIDADES BÁSICAS
# ==========================================
class Utils:
    """Utilidades reutilizables"""

    @staticmethod
    def cargar_imagenes(ruta: str) -> List[str]:
        """Carga lista de imágenes"""
        path = Path(ruta)
        extensiones = (".jpg", ".png", ".jpeg")
        imagenes = [str(p) for p in path.glob("**/*") if p.suffix.lower() in extensiones]
        print(f"Encontradas {len(imagenes)} imágenes")
        return imagenes

    @staticmethod
    def preparar_imagen(ruta: str, max_size: int = 1024) -> Image.Image:
        """Prepara imagen para procesamiento"""
        img = Image.open(ruta).convert("RGB")
        if max(img.size) > max_size:
            img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
        return img

    @staticmethod
    def calcular_hash(ruta: str) -> str:
        """Calcula hash único de archivo"""
        return hashlib.md5(open(ruta, 'rb').read()).hexdigest()[:8]

    @staticmethod
    def guardar_json(datos: Any, archivo: str) -> None:
        """Guarda datos en JSON"""
        with open(archivo, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)
        print(f"Guardado: {archivo}")

print("Utilidades definidas")

Utilidades definidas


In [30]:
# ==========================================
# 3: DETECTOR CORE
# ==========================================
class DetectorPersonas:
    """Detector principal - Clase única y enfocada"""

    def __init__(self, modelo_name: str):
        self.modelo_name = modelo_name
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        print(f"🤖 Cargando: {modelo_name}")
        self.processor = AutoImageProcessor.from_pretrained(modelo_name)
        self.model = AutoModelForUniversalSegmentation.from_pretrained(modelo_name)
        self.model.to(self.device)
        self.model.eval()
        print(f"Modelo cargado en {self.device}")

    def detectar_en_imagen(self, imagen: Image.Image, umbrales: List[float]) -> Dict:
        """Detecta personas con múltiples umbrales"""
        h, w = imagen.size[1], imagen.size[0]  # PIL format

        # Inferencia una sola vez
        inputs = self.processor(images=imagen, return_tensors="pt").to(self.device)

        inicio = time.time()
        with torch.no_grad():
            outputs = self.model(**inputs)
        tiempo_ms = (time.time() - inicio) * 1000

        # Probar cada umbral
        resultados = {'tiempo_inferencia_ms': tiempo_ms}

        for umbral in umbrales:
            try:
                resultado = self.processor.post_process_instance_segmentation(
                    outputs, target_sizes=[(h, w)], threshold=umbral
                )[0]

                # Extraer información
                labels = resultado.get("labels", [])
                scores = resultado.get("scores", [])

                # Contar personas (clase 0) y otras detecciones
                personas = sum(1 for l in labels if int(l.item()) == 0)
                total_detecciones = len(labels)

                # Scores de personas
                scores_personas = [
                    float(s.item()) for l, s in zip(labels, scores)
                    if int(l.item()) == 0
                ]

                # Todas las clases detectadas
                todas_clases = [int(l.item()) for l in labels]
                todos_scores = [float(s.item()) for s in scores]

                resultados[f'umbral_{umbral}'] = {
                    'personas': personas,
                    'total': total_detecciones,
                    'scores_personas': scores_personas,
                    'todas_clases': todas_clases,
                    'todos_scores': todos_scores,
                    'max_score': max(todos_scores) if todos_scores else 0.0
                }

            except Exception as e:
                resultados[f'umbral_{umbral}'] = {
                    'error': str(e),
                    'personas': 0,
                    'total': 0
                }

        return resultados

    def limpiar(self):
        """Limpia memoria"""
        del self.model, self.processor
        torch.cuda.empty_cache()

print("DetectorPersonas definido")

DetectorPersonas definido


In [31]:
# ==========================================
# 4: PROCESADOR DE RESULTADOS
# ==========================================
class ProcesadorResultados:
    """Procesa y formatea resultados"""

    def __init__(self, output_path: str):
        self.output_path = Path(output_path)

    def procesar_imagen(self, ruta_imagen: str, detector: DetectorPersonas,
                       umbrales: List[float]) -> Dict:
        """Procesa una imagen completa"""
        inicio = time.time()

        try:
            # Preparar imagen
            imagen = Utils.preparar_imagen(ruta_imagen)
            hash_img = Utils.calcular_hash(ruta_imagen)

            # Información básica
            img_array = np.array(imagen)
            gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

            info_imagen = {
                'archivo': os.path.basename(ruta_imagen),
                'hash': hash_img,
                'tamaño': imagen.size,
                'brillo_promedio': float(np.mean(gray))
            }

            # Detectar
            resultados_deteccion = detector.detectar_en_imagen(imagen, umbrales)

            # Resultado final
            tiempo_total = (time.time() - inicio) * 1000

            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': info_imagen,
                'deteccion': resultados_deteccion,
                'tiempo_total_ms': tiempo_total,
                'modelo': detector.modelo_name,
                'exitoso': True
            }

        except Exception as e:
            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': {'archivo': os.path.basename(ruta_imagen)},
                'error': str(e),
                'exitoso': False
            }

    def mostrar_resumen(self, resultados: List[Dict]) -> None:
        """Muestra resumen de resultados"""
        exitosos = sum(1 for r in resultados if r.get('exitoso', False))
        print(f"\nRESUMEN: {exitosos}/{len(resultados)} imágenes procesadas")

        if exitosos == 0:
            return

        # Estadísticas por umbral
        for umbral in config.UMBRALES['agresivo']:
            personas_detectadas = []
            for r in resultados:
                if r.get('exitoso', False):
                    deteccion = r.get('deteccion', {})
                    umbral_data = deteccion.get(f'umbral_{umbral}', {})
                    personas_detectadas.append(umbral_data.get('personas', 0))

            if personas_detectadas:
                total_personas = sum(personas_detectadas)
                imagenes_con_personas = sum(1 for p in personas_detectadas if p > 0)
                print(f"  Umbral {umbral:5.3f}: {total_personas} personas en {imagenes_con_personas} imágenes")

print("ProcesadorResultados definido")

ProcesadorResultados definido


In [32]:
# ==========================================
# 5: FUNCIONES EJECUTABLES
# ==========================================

def ejecutar_evaluacion_basica(modelo_idx: int = 0, umbral_config: str = 'agresivo'):
    """
    FUNCIÓN EJECUTABLE: Evaluación básica

    Args:
        modelo_idx: Índice del modelo (0, 1, 2)
        umbral_config: Configuración de umbrales ('conservador', 'normal', 'agresivo', 'ultra')
    """
    print(f"EVALUACIÓN BÁSICA - Modelo {modelo_idx}, Umbrales: {umbral_config}")
    print("=" * 60)

    # Validar parámetros
    if modelo_idx >= len(config.MODELOS):
        print(f"Modelo index inválido. Máximo: {len(config.MODELOS)-1}")
        return

    if umbral_config not in config.UMBRALES:
        print(f"Configuración inválida. Disponibles: {list(config.UMBRALES.keys())}")
        return

    # Cargar datos
    imagenes = Utils.cargar_imagenes(config.DATASET_PATH)
    if not imagenes:
        print("No hay imágenes")
        return

    # Ejecutar
    modelo = config.MODELOS[modelo_idx]
    umbrales = config.UMBRALES[umbral_config]

    detector = DetectorPersonas(modelo)
    procesador = ProcesadorResultados(config.OUTPUT_PATH)

    # Procesar cada imagen
    resultados = []
    for i, ruta in enumerate(tqdm(imagenes, desc="Procesando")):
        print(f"[{i+1:3d}/{len(imagenes)}] {os.path.basename(ruta)}")
        resultado = procesador.procesar_imagen(ruta, detector, umbrales)
        resultados.append(resultado)

        # Log básico
        if resultado.get('exitoso', False):
            deteccion = resultado['deteccion']
            mejor_umbral = min(umbrales)  # Usar el más sensible
            datos = deteccion.get(f'umbral_{mejor_umbral}', {})
            personas = datos.get('personas', 0)
            total = datos.get('total', 0)
            print(f"{personas} personas de {total} detecciones")
        else:
            print(f"Error: {resultado.get('error', 'unknown')}")

    # Guardar y mostrar resumen
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archivo = config.OUTPUT_PATH / f"evaluacion_{modelo_idx}_{umbral_config}_{timestamp}.json"
    Utils.guardar_json(resultados, str(archivo))

    procesador.mostrar_resumen(resultados)
    detector.limpiar()

    return str(archivo)

def probar_imagen_individual(ruta_imagen: str, mostrar_detalles: bool = True):
    """
    FUNCIÓN EJECUTABLE: Probar una imagen específica

    Args:
        ruta_imagen: Ruta completa a la imagen
        mostrar_detalles: Si mostrar información detallada
    """
    print(f"PRUEBA INDIVIDUAL: {os.path.basename(ruta_imagen)}")
    print("=" * 60)

    if not os.path.exists(ruta_imagen):
        print(f"Imagen no encontrada: {ruta_imagen}")
        return

    # Usar modelo más potente y umbrales ultra sensibles
    detector = DetectorPersonas(config.MODELOS[0])  # El más grande
    procesador = ProcesadorResultados(config.OUTPUT_PATH)

    resultado = procesador.procesar_imagen(ruta_imagen, detector, config.UMBRALES['ultra'])

    if not resultado.get('exitoso', False):
        print(f"Error: {resultado.get('error')}")
        detector.limpiar()
        return

    # Mostrar resultados
    deteccion = resultado['deteccion']
    imagen_info = resultado['imagen']

    print(f"Imagen: {imagen_info['tamaño']} pixels, brillo: {imagen_info['brillo_promedio']:.1f}")
    print(f"Tiempo: {deteccion['tiempo_inferencia_ms']:.1f}ms")
    print()

    # Resultados por umbral
    print("DETECCIONES POR UMBRAL:")
    for umbral in config.UMBRALES['ultra']:
        datos = deteccion.get(f'umbral_{umbral}', {})
        if 'error' in datos:
            print(f"  {umbral:6.3f}: Error - {datos['error']}")
            continue

        personas = datos.get('personas', 0)
        total = datos.get('total', 0)
        max_score = datos.get('max_score', 0)

        print(f"  {umbral:6.3f}: {personas:2d} personas de {total:2d} total (max score: {max_score:.4f})")

        if mostrar_detalles and datos.get('todas_clases'):
            clases_unicas = sorted(set(datos['todas_clases']))
            print(f"             Clases: {clases_unicas}")

    # Recomendación
    mejor_umbral_data = deteccion.get('umbral_0.01', deteccion.get('umbral_0.001', {}))
    personas_detectadas = mejor_umbral_data.get('personas', 0)

    print(f"\nRESULTADO: {personas_detectadas} personas detectadas")
    if personas_detectadas == 0:
        total_detecciones = mejor_umbral_data.get('total', 0)
        if total_detecciones > 0:
            print(" Se detectaron otros objetos pero no personas")
            print("   Posibles soluciones:")
            print("   - Usar umbrales aún más bajos")
            print("   - Verificar que la imagen contenga personas visibles")
            print("   - Probar con otro modelo")
        else:
            print(" No se detectó ningún objeto")

    detector.limpiar()
    return resultado

def comparar_modelos_rapido(max_imagenes: int = 3):
    """
    FUNCIÓN EJECUTABLE: Comparación rápida de modelos

    Args:
        max_imagenes: Número máximo de imágenes a procesar
    """
    print(f"⚖️ COMPARACIÓN RÁPIDA - {max_imagenes} imágenes máximo")
    print("=" * 60)

    imagenes = Utils.cargar_imagenes(config.DATASET_PATH)[:max_imagenes]
    if not imagenes:
        print("No hay imágenes")
        return

    umbrales = config.UMBRALES['agresivo']
    resultados_comparacion = {}

    for i, modelo in enumerate(config.MODELOS):
        print(f"\nModelo {i+1}/{len(config.MODELOS)}: {modelo.split('/')[-1]}")

        detector = DetectorPersonas(modelo)
        procesador = ProcesadorResultados(config.OUTPUT_PATH)

        resultados_modelo = []
        for ruta in imagenes:
            resultado = procesador.procesar_imagen(ruta, detector, umbrales)
            resultados_modelo.append(resultado)

        # Estadísticas rápidas
        exitosos = sum(1 for r in resultados_modelo if r.get('exitoso', False))

        personas_totales = 0
        tiempo_promedio = 0

        for resultado in resultados_modelo:
            if resultado.get('exitoso', False):
                # Usar umbral más sensible para contar
                deteccion = resultado['deteccion']
                umbral_data = deteccion.get('umbral_0.01', deteccion.get('umbral_0.05', {}))
                personas_totales += umbral_data.get('personas', 0)
                tiempo_promedio += resultado.get('tiempo_total_ms', 0)

        if exitosos > 0:
            tiempo_promedio /= exitosos

        resultados_comparacion[modelo] = {
            'exitosos': exitosos,
            'personas_totales': personas_totales,
            'tiempo_promedio_ms': tiempo_promedio
        }

        print(f" {exitosos}/{len(imagenes)} exitosas")
        print(f" {personas_totales} personas detectadas")
        print(f" {tiempo_promedio:.1f}ms promedio")

        detector.limpiar()

    # Resumen final
    print(f"\nRESUMEN COMPARACIÓN:")
    print("-" * 40)
    for modelo, datos in resultados_comparacion.items():
        nombre = modelo.split('/')[-1]
        print(f"{nombre:30s}: {datos['personas_totales']:2d} personas, {datos['tiempo_promedio_ms']:5.1f}ms")

    return resultados_comparacion

print("Funciones ejecutables definidas")

Funciones ejecutables definidas


In [34]:
## Ejecutamos evaluación básica.

for i in range(len(config.MODELOS)):
    for umbral in config.UMBRALES.keys():
        ejecutar_evaluacion_basica(i, umbral)

EVALUACIÓN BÁSICA - Modelo 0, Umbrales: conservador
Encontradas 1 imágenes
🤖 Cargando: facebook/mask2former-swin-large-coco-instance
Modelo cargado en cuda


Procesando:   0%|          | 0/1 [00:00<?, ?it/s]

[  1/1] _DSC0147.jpg


Procesando: 100%|██████████| 1/1 [00:01<00:00,  1.48s/it]

0 personas de 0 detecciones
Guardado: /content/drive/MyDrive/TFM/mask2former/resultados/evaluacion_0_conservador_20250901_191543.json

RESUMEN: 1/1 imágenes procesadas
  Umbral 0.010: 0 personas en 0 imágenes
  Umbral 0.050: 0 personas en 0 imágenes
  Umbral 0.100: 0 personas en 0 imágenes
  Umbral 0.300: 0 personas en 0 imágenes
EVALUACIÓN BÁSICA - Modelo 0, Umbrales: normal
Encontradas 1 imágenes
🤖 Cargando: facebook/mask2former-swin-large-coco-instance


...

Modelo cargado en cuda


Procesando:   0%|          | 0/1 [00:00<?, ?it/s]

[  1/1] _DSC0147.jpg


Procesando: 100%|██████████| 1/1 [00:00<00:00,  2.72it/s]

0 personas de 0 detecciones
Guardado: /content/drive/MyDrive/TFM/mask2former/resultados/evaluacion_2_ultra_20250901_191603.json

RESUMEN: 1/1 imágenes procesadas
  Umbral 0.010: 0 personas en 0 imágenes
  Umbral 0.050: 0 personas en 0 imágenes
  Umbral 0.100: 0 personas en 0 imágenes
  Umbral 0.300: 0 personas en 0 imágenes
